# Project Group 11

Members:  Alessandro Casati (6544649),  Bas van den Muijsenberg (5797578) , Hylke Bleeker (5589355), Jan-Pieter Vermeer (6340261),  Mike Geerts (6276210)


# Research Objective

## Analysis and delay prediction of flights across top 20 airports in Europe in 2023-24

**Research Question**  
What factors drive airport delays across the top 10 European airports in 2023-24, and can we build predictive models to identify possible delays for a flight using the information available?

**Sub Questions**
- What type of delays are occurring most at the airports?
- Which airports experience the highest, the lowest delay, and most consistent delay rates?
- How do delays relate to traffic volume and efficiency?
- How do delays distribute geographically across Europe, and which airports stand out as persistent “delay hotspots”?
- Which features (traffic levels, weekday/weekend, month, etc.) are the strongest predictors of high-delay days?
- What model can predict the chance of delay and the amount delay for a flight?

# Contribution Statement

*Be specific. Some of the tasks can be coding (expect everyone to do this), background research, conceptualisation, visualisation, data analysis, data modelling*

**Author 1**:

**Author 2**:

**Author 3**:

# Data Used

## Datasets
| Dataset | Dataset description (function) | Columns needed |
|---|---|---|
| Airport Traffic | Airport capacity of inbound and outbound flights for network manager and airport operator | All |
| Airport ATFM_Delay | The Airport Arrival ATFM Delay provides an indication of ATFM delays on the ground due to constraints at airports | FLT_DATE, APT_ICAO (to merge DB) + all the remaining types of delays |
| Airport Punctuality | Daily performance tracker of airport punctuality, delays, and operational efficiency | All + add APT_ICAO for merging |
| All_Pre_Departure_Delays | Daily overview of total pre-departure delay in minutes per airport that includes all delay causes | All |
| ATC_Pre_Departure_Delay | Proxy for ATC induced delays at the departure stand (IATA delay code 89) | Pending (depends on data analysis findings) |
| Additional taxi-out time | Taxi time from gate to runway and associated delay | All |
| Additional taxi-in time | Taxi time from runway to gate and associated delay | All |

## Information extracted from datasets
| Dataset | Key Information |
|---|---|
| Airport Traffic | IFR arrivals, departures for Airport Operator and Network Manager |
| Airport ATFM_Delay | Disruption type and count, number of delayed arrival flights |
| Airport Punctuality | Arrival & departure punctuality %, avg delay (dep & arr), operated schedules % |
| Pre_Departure_Delay | AO departures, AO pre-departure delay (minutes) |
| Additional taxi-out time | Number of flights with available data + total taxi in time |
| Additional taxi-in time | Number of flights with available data + total taxi out time |

# Data Pipeline

## Data Exploration & Cleaning
- Import & clean the datasets: remove missing/invalid values, select only useful columns and merge.
- Create basic delay metrics:
  - Delay categories (On time, Delayed, Delayed15)
  - Day category (Weekend, Weekdays)

## Descriptive Data Analysis
- Time patterns: heatmaps/line charts/stacked charts (delays by day, month, cause).
- Geospatial view: interactive maps color-coded by average airport delay.
- Efficiency scatterplots: traffic volume vs. % delay

## Predictive Modeling
- Predict delay occurrence for a flight using departure airport and its performances, average airport delay, weekday/weekend & month.
- Models: Logistic Regression and/or Random Forest.
- Testing with 2025 data (unseen during training).

## Clustering & Patterns
- Cluster airports by daily delay patterns (e.g., “always congested,” “seasonal peaks,” “mostly smooth”).

# Data Cleaning

In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


## TO-DO list, by Wed 8/10
| Dataset | Key Information | Assigned to |
|---|---|---|
| Airport Traffic | Imported + top20 defined: missing total number of departures and arrivals | Ale |
| Airport ATFM_Delay | Extract top20 aiports by comparing it with df_top10 and overall and sum for all the 2 years | Bas |
| Airport Punctuality | Filter for 23-24, find top 20, compare with df_top20 (some airports might be missing), get all the columns | Mike |
| Pre_Departure_Delay | TBD. is the data actually useful for our RQ, if yes extract top20 + last 5 columns | JP |
| Additional taxi-out time | Extract top20 airports + TF, TOT_REF_NB_FL, TOT_REF&ADD_TIME_MIN, PIVOT_LABEL | Hylke |
| Additional taxi-in time | Extract top20 airports + TF, TOT_REF_NB_FL, TOT_REF&ADD_TIME_MIN, PIVOT_LABEL | Hylke |

Some comments: for now we focus on the totals for 2023-24 (combined). Watch out that some datasets don't have matching columns and you can't merge them together, you might need to compare them manually or use other references (especially for Punctuality dataset)

## Airport Traffic
Importing dataset

In [49]:
df_AT = pd.read_excel("datasets/AirportTraffic.xlsx")
df_TIn = pd.read_excel("datasets/Taxi-In_Additional_Time.xlsx")
df_TOut = pd.read_excel("datasets/Taxi-Out_Additional_Time.xlsx")
df_AAD = pd.read_excel("datasets/AA_ATFM_Delay.xlsx")
df_AP = pd.read_excel("datasets/Airports_Punctuality.xlsx")

Find top 20 airports by total flights and create a new dataframe with key data

In [56]:
df_top20 = (
    df_AT.groupby("APT_ICAO")[["FLT_TOT_1", "FLT_DEP_1", "FLT_ARR_1"]] #Group by airport code
    .sum().sort_values(by="FLT_TOT_1",ascending=False) #Sum the values for each code of the 3 columns indicated
    .head(20).reset_index()) #Change "20" to change the number of airports analysed
# Adding airport's city name and state from original dataset
df_top20 = (df_top20.merge(df_AT[["APT_ICAO", "APT_NAME", "STATE_NAME"]]
                           .drop_duplicates(), on="APT_ICAO", how="left"))
df_top20.head(20)

,APT_ICAO,FLT_TOT_1,FLT_DEP_1,FLT_ARR_1,APT_NAME,STATE_NAME
0,LTFM,1014869,507380,507489,Istanbul,Türkiye
1,EHAM,947254,473657,473597,Amsterdam - Schiphol,Netherlands
2,EGLL,933098,466473,466625,London - Heathrow,United Kingdom
3,LFPG,921916,460997,460919,Paris-Charles-de-Gaulle,France
4,EDDF,871219,435565,435654,Frankfurt,Germany
5,LEMD,809535,404744,404791,Madrid - Barajas,Spain
6,LEBL,666969,333465,333504,Barcelona,Spain
7,EDDM,624316,312141,312175,Munich,Germany
8,LIRF,582158,291068,291090,Rome - Fiumicino,Italy
9,EGKK,522331,261146,261185,London - Gatwick,United Kingdom


# Airport ATFM_Delay

Delay for every day and all top 20 airports.
Removing Na, and only top 20 airports, sorted by airport traffic

In [ ]:
df_AAD = df_AAD.dropna()
df_AAD_top20 = df_AAD[df_AAD["APT_ICAO"].isin(df_top20["APT_ICAO"])]
df_AAD_top20

#Lists of code and names of top20 airports
airports_name_list = df_top20["APT_NAME"].tolist()
airports_code_list = df_top20["APT_ICAO"].tolist()

#Dataframe of codes, useful for merging 
df_airport_codes = pd.DataFrame({'APT_ICAO': airports_code_list})

df_AAD_top20 = df_AAD_top20.set_index("APT_ICAO").loc[airports_code_list].reset_index()


df_AAD_top20.head()

,APT_ICAO,YEAR,MONTH_NUM,MONTH_MON,FLT_DATE,APT_NAME,STATE_NAME,FLT_ARR_1,DLY_APT_ARR_1,DLY_APT_ARR_A_1,...,DLY_APT_ARR_R_1,DLY_APT_ARR_S_1,DLY_APT_ARR_T_1,DLY_APT_ARR_V_1,DLY_APT_ARR_W_1,DLY_APT_ARR_NA_1,FLT_ARR_1_DLY,FLT_ARR_1_DLY_15,ATFM_VERSION,Pivot Label
0,LTFM,2023,1,JAN,44928,Istanbul,Türkiye,676,1720.0,0.0,...,0.0,0.0,0.0,0.0,1720.0,0.0,83.0,38.0,v2,Istanbul (LTFM)
1,LTFM,2023,1,JAN,44938,Istanbul,Türkiye,607,665.0,0.0,...,0.0,0.0,0.0,0.0,665.0,0.0,34.0,19.0,v2,Istanbul (LTFM)
2,LTFM,2023,1,JAN,44939,Istanbul,Türkiye,635,78.0,0.0,...,0.0,0.0,0.0,0.0,78.0,0.0,5.0,2.0,v2,Istanbul (LTFM)
3,LTFM,2023,1,JAN,44948,Istanbul,Türkiye,646,21.0,0.0,...,0.0,0.0,0.0,0.0,21.0,0.0,3.0,0.0,v2,Istanbul (LTFM)
4,LTFM,2023,1,JAN,44954,Istanbul,Türkiye,641,351.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,30.0,6.0,v2,Istanbul (LTFM)


Total delay for every airport 

In [42]:
ATFM_cols = df_AAD.columns[7:27].tolist() #Columns neded

df_AAD_total_delays_per_airport = (
    df_AAD_top20[df_AAD_top20["APT_ICAO"].isin(airports_code_list)].groupby(["APT_ICAO", "APT_NAME", "STATE_NAME"])[ATFM_cols]
    .sum()
    .reset_index()
)

order = df_top20["APT_ICAO"].tolist()
df_AAD_total_delays_per_airport = df_AAD_total_delays_per_airport.set_index("APT_ICAO").loc[order].reset_index() #Do we care for the order?

df_AAD_total_delays_per_airport

,APT_ICAO,APT_NAME,STATE_NAME,FLT_ARR_1,DLY_APT_ARR_1,DLY_APT_ARR_A_1,DLY_APT_ARR_C_1,DLY_APT_ARR_D_1,DLY_APT_ARR_E_1,DLY_APT_ARR_G_1,...,DLY_APT_ARR_O_1,DLY_APT_ARR_P_1,DLY_APT_ARR_R_1,DLY_APT_ARR_S_1,DLY_APT_ARR_T_1,DLY_APT_ARR_V_1,DLY_APT_ARR_W_1,DLY_APT_ARR_NA_1,FLT_ARR_1_DLY,FLT_ARR_1_DLY_15
0,LTFM,Istanbul,Türkiye,75507,230592.0,0.0,311.0,0.0,0.0,19684.0,...,0.0,2706.0,0.0,0.0,1415.0,0.0,206476.0,0.0,6800.0,4442.0
1,EHAM,Amsterdam - Schiphol,Netherlands,433799,1540691.0,0.0,12665.0,0.0,6204.0,650129.0,...,7292.0,18581.0,0.0,591.0,4280.0,0.0,840949.0,0.0,93661.0,29292.0
2,EGLL,London - Heathrow,United Kingdom,336787,1230954.0,5624.0,96191.0,0.0,4202.0,35423.0,...,6433.0,4818.0,0.0,14679.0,741.0,0.0,1062843.0,0.0,65314.0,28145.0
3,LFPG,Paris-Charles-de-Gaulle,France,135569,119040.0,24.0,763.0,0.0,241.0,1152.0,...,265.0,10259.0,0.0,9306.0,3704.0,0.0,85256.0,0.0,6960.0,2882.0
4,EDDF,Frankfurt,Germany,186063,563486.0,493.0,10253.0,0.0,9417.0,177461.0,...,14543.0,1445.0,0.0,7450.0,18717.0,0.0,323690.0,0.0,27388.0,13731.0
5,LEMD,Madrid - Barajas,Spain,294867,278691.0,1530.0,95450.0,0.0,0.0,47110.0,...,3723.0,20656.0,0.0,0.0,1858.0,1032.0,107332.0,0.0,18858.0,5932.0
6,LEBL,Barcelona,Spain,136472,384152.0,1079.0,5123.0,0.0,960.0,16489.0,...,0.0,523.0,0.0,0.0,3889.0,31802.0,324287.0,0.0,15815.0,7609.0
7,EDDM,Munich,Germany,70404,108797.0,25.0,2193.0,0.0,950.0,5265.0,...,0.0,1794.0,0.0,464.0,263.0,0.0,97843.0,0.0,5032.0,2348.0
8,LIRF,Rome - Fiumicino,Italy,24871,39145.0,964.0,0.0,0.0,0.0,8431.0,...,4829.0,773.0,0.0,0.0,0.0,0.0,22789.0,0.0,2222.0,959.0
9,EGKK,London - Gatwick,United Kingdom,191655,1032548.0,0.0,32521.0,0.0,239.0,363605.0,...,7075.0,0.0,0.0,205442.0,1594.0,0.0,422072.0,0.0,38960.0,22324.0


Mean delay

In [59]:
ATFM_cols = df_AAD.columns[7:27].tolist() #Columns neded

df_AAD_mean_delays_per_airport = (
    df_AAD_top20[df_AAD_top20["APT_ICAO"].isin(airports_code_list)].groupby(["APT_ICAO", "APT_NAME", "STATE_NAME"])[ATFM_cols]
    .mean()
    .reset_index()
)

order = df_top20["APT_ICAO"].tolist()
df_AAD_mean_delays_per_airport = df_AAD_mean_delays_per_airport.set_index("APT_ICAO").loc[order].reset_index() #Do we care for the order?

df_AAD_mean_delays_per_airport

,APT_ICAO,APT_NAME,STATE_NAME,FLT_ARR_1,DLY_APT_ARR_1,DLY_APT_ARR_A_1,DLY_APT_ARR_C_1,DLY_APT_ARR_D_1,DLY_APT_ARR_E_1,DLY_APT_ARR_G_1,...,DLY_APT_ARR_O_1,DLY_APT_ARR_P_1,DLY_APT_ARR_R_1,DLY_APT_ARR_S_1,DLY_APT_ARR_T_1,DLY_APT_ARR_V_1,DLY_APT_ARR_W_1,DLY_APT_ARR_NA_1,FLT_ARR_1_DLY,FLT_ARR_1_DLY_15
0,LTFM,Istanbul,Türkiye,686.427273,2096.290909,0.000000,2.827273,0.0,0.000000,178.945455,...,0.000000,24.600000,0.0,0.000000,12.863636,0.000000,1877.054545,0.0,61.818182,40.381818
1,EHAM,Amsterdam - Schiphol,Netherlands,657.271212,2334.380303,0.000000,19.189394,0.0,9.400000,985.043939,...,11.048485,28.153030,0.0,0.895455,6.484848,0.000000,1274.165152,0.0,141.910606,44.381818
2,EGLL,London - Heathrow,United Kingdom,643.952199,2353.640535,10.753346,183.921606,0.0,8.034417,67.730402,...,12.300191,9.212237,0.0,28.066922,1.416826,0.000000,2032.204589,0.0,124.883365,53.814532
3,LFPG,Paris-Charles-de-Gaulle,France,633.500000,556.261682,0.112150,3.565421,0.0,1.126168,5.383178,...,1.238318,47.939252,0.0,43.485981,17.308411,0.000000,398.392523,0.0,32.523364,13.467290
4,EDDF,Frankfurt,Germany,604.100649,1829.500000,1.600649,33.288961,0.0,30.574675,576.172078,...,47.217532,4.691558,0.0,24.188312,60.769481,0.000000,1050.941558,0.0,88.922078,44.581169
5,LEMD,Madrid - Barajas,Spain,557.404537,526.826087,2.892250,180.434783,0.0,0.000000,89.054820,...,7.037807,39.047259,0.0,0.000000,3.512287,1.950851,202.896030,0.0,35.648393,11.213611
6,LEBL,Barcelona,Spain,485.665480,1367.088968,3.839858,18.231317,0.0,3.416370,58.679715,...,0.000000,1.861210,0.0,0.000000,13.839858,113.174377,1154.046263,0.0,56.281139,27.078292
7,EDDM,Munich,Germany,445.594937,688.588608,0.158228,13.879747,0.0,6.012658,33.322785,...,0.000000,11.354430,0.0,2.936709,1.664557,0.000000,619.259494,0.0,31.848101,14.860759
8,LIRF,Rome - Fiumicino,Italy,428.810345,674.913793,16.620690,0.000000,0.0,0.000000,145.362069,...,83.258621,13.327586,0.0,0.000000,0.000000,0.000000,392.913793,0.0,38.310345,16.534483
9,EGKK,London - Gatwick,United Kingdom,383.310000,2065.096000,0.000000,65.042000,0.0,0.478000,727.210000,...,14.150000,0.000000,0.0,410.884000,3.188000,0.000000,844.144000,0.0,77.920000,44.648000


# Taxi In & Out

In [93]:
df_taxi_time_in = (
    df_TIn[(df_TIn["APT_ICAO"].isin(airports_code_list)) & 
           ((df_TIn["YEAR"] == 2023) | (df_TIn["YEAR"] == 2024))]
    .groupby("APT_ICAO")[["VALID_FL", "TOTAL_REF_NB_FL", "TOTAL_REF_TIME_MIN", "TOTAL_ADD_TIME_MIN"]]
    .sum()
    .reset_index()
)
#In order: filter only airports in top 20, filter only values from 2023-24, group by code and sum the values in the 
#listed columns, reset the index to have the codes as a column itselft (useful for merging)

# Rename the columns to for in values
in_cols_to_rename = {
    "VALID_FL": "VALID_FL_IN",
    "TOTAL_REF_NB_FL": "TOTAL_REF_NB_FL_IN",
    "TOTAL_REF_TIME_MIN": "TOT_REF_TIME_MIN_IN",
    "TOTAL_ADD_TIME_MIN": "TOT_ADD_TIME_MIN_IN"
}
df_taxi_time_in = df_taxi_time_in.rename(columns=in_cols_to_rename)

# Merging taxi with airport codes
df_taxi_time_final = pd.merge(df_airport_codes, df_taxi_time_in, on='APT_ICAO', how='left')


In [ ]:
df_taxi_time_out = (
    df_TOut[(df_TOut["APT_ICAO"].isin(airports_code_list)) & 
           ((df_TOut["YEAR"] == 2023) | (df_TOut["YEAR"] == 2024))]
    .groupby("APT_ICAO")[["VALID_FL", "TOTAL_REF_NB_FL", "TOTAL_REF_TIME_MIN", "TOTAL_ADD_TIME_MIN"]]
    .sum()
    .reset_index()
)
#Same operations as df_taxi_time_in

# Rename the columns to for out values
out_cols_to_rename = {
    "VALID_FL": "VALID_FL_OUT",
    "TOTAL_REF_NB_FL": "TOTAL_REF_NB_FL_OUT",
    "TOTAL_REF_TIME_MIN": "TOT_REF_TIME_MIN_OUT",
    "TOTAL_ADD_TIME_MIN": "TOT_ADD_TIME_MIN_OUT"
}
df_taxi_time_out = df_taxi_time_out.rename(columns=out_cols_to_rename)

# Merging taxi in with taxi out
df_taxi_time = pd.merge(df_taxi_time_final, df_taxi_time_out, on='APT_ICAO', how='left')

print(df_taxi_time.head())

# Airport Punctionality

In [100]:
airport_map = {
    "LTFM": "Istanbul",
    "EHAM": "Amsterdam",
    "EGLL": "London Heathrow",
    "LFPG": "Paris Charles de Gaulle",
    "EDDF": "Frankfurt",
    "LEMD": "Madrid Barajas",
    "LEBL": "Barcelona",
    "EDDM": "Munich",
    "LIRF": "Rome Fiumicino",
    "EGKK": "London Gatwick",
    "LSZH": "Zurich",
    "LGAV": "Athens",
    "EIDW": "Dublin",
    "LOWW": "Vienna",
    "LEPA": "Palma de Mallorca",
    "EKCH": "Copenhagen",
    "LPPT": "Lisbon",
    "LTFJ": "Istanbul Sabiha Gokcen",
    "LTAI": "Antalya",
    "ENGM": "Oslo"
}

top20_codes = [airport_map[a] for a in df_top20['APT_ICAO'] if a in airport_map]
df_AP_top20 = df_AP[df_AP['Airport'].isin(top20_codes)]

name_to_icao = {name: icao for icao, name in airport_map.items()}
icao_codes = df_AP_top20["Airport"].map(name_to_icao)
df_AP_top20.insert(loc=2, column="APT_ICAO", value=df_AP_top20["Airport"].map(name_to_icao))


df_AP_top20

,Date,Airport,APT_ICAO,Departure Punctuality %,Arrival Punctuality %,Avg Departure Schedule Delay,Avg Arrival Schedule Delay,Avg Departure - Arrival Schedule Delay,Operated Schedules %
1367,2022-01-01,Frankfurt,EDDF,0.818898,0.804217,13.043920,13.426908,-0.382988,0.922170
1368,2022-01-02,Frankfurt,EDDF,0.707216,0.713389,17.652062,18.506590,-0.854528,0.946463
1369,2022-01-03,Frankfurt,EDDF,0.738938,0.755991,16.235140,13.956209,2.278931,0.952183
1370,2022-01-04,Frankfurt,EDDF,0.630542,0.725962,16.605624,14.621955,1.983669,0.953811
1371,2022-01-05,Frankfurt,EDDF,0.782816,0.821951,10.382379,9.699187,0.683192,0.946785
...,...,...,...,...,...,...,...,...,...
43735,2025-09-24,Istanbul,LTFM,0.840440,0.831309,8.972031,11.150967,-2.178936,0.922894
43736,2025-09-25,Istanbul,LTFM,0.794157,0.843750,11.858544,8.783967,3.074576,0.930314
43737,2025-09-26,Istanbul,LTFM,0.713147,0.770039,15.778220,13.545795,2.232425,0.938824
43738,2025-09-27,Istanbul,LTFM,0.748021,0.782377,17.316623,13.748999,3.567624,0.948931
